In [ ]:
import numpy as np
import openmc
import openmc.checkvalue as cv
from openmc.checkvalue import PathLike
from openmc.utility_funcs import change_directory
from openmc._xml import get_text
from openmc.mixin import IDManagerMixin
from openmc.surface import _BOUNDARY_TYPES
from openmc.utility_funcs import input_path
import warnings
from abc import ABC, abstractmethod
from collections.abc import Iterable, Sequence, Mapping
from functools import wraps
from math import pi, sqrt, atan2
from numbers import Integral, Real
import h5py
import lxml.etree as ET
import numpy as np
from pathlib import Path

In [ ]:
with open("MCNP_source.inp", "r") as f:
    lines = f.readlines()
SP_lines = []
SI_lines = []
DS3Q_lines= []
SP2_lines = [0]

is_sp_card = False
SP_start=False
SI_start=False
DS3Q_start=False
SP2_start=False
for line in lines:
    
    
    if(DS3Q_start):
        if(line.find("C")==-1):
        
            line=line.strip()
            for i, word in enumerate(line.split()):
                if(i%2==0):
                    DS3Q_lines.append(word)    
        else:
            DS3Q_start = False

    if(line.find("DS3 Q")!=-1):
        DS3Q_start = True

    if(line.find("SP500 D 0 ")!=-1):
        SP_start=True

    if(SI_start):
        if(line.find("C")==-1):
            line=line.strip()
            [SI_lines.append(word) for word in line.split()] 
        else:
            SI_start = False

    if(line.find("SI500 H")!=-1):
        SI_start = True


    if(SP2_start):
        if(line.find("C")==-1):
            line=line.strip()
            [SP2_lines.append(word) for word in line.split()] 
        else:
            SP2_start = False

    if(line.find("SP2 D 0 ")!=-1):
        SP2_start = True
        
    if(SP_start):
        if(line.find("C")!=-1): #if C is not not found (if C is found)
            is_sp_card = False
            # print(line)
        if(is_sp_card):
            line=line.strip()
            [SP_lines[-1].append(word) for word in line.split()]            
        if(line.find("SP")!=-1): #if SP is not not found
            # print(line)
            if(len(SP_lines) !=0):
                if(len(SP_lines[-1])!=211):
                    print("data list not 211 long")
            SP_lines.append([0]) #Since SI has 211 entries, I assume the first 0 after SP D is an entry
            is_sp_card = True

In [ ]:
height_displacement=-100
r_margin=1.4/2
z_margin=0.95/2
int_weight=0

weight_list=[]
z_list=[]
r_list=[]

for DSR_index, rad in enumerate(DS3Q_lines):
    for SI_index, height in enumerate(SI_lines):
        int_weight+=float(SP_lines[DSR_index][SI_index])*float(SP2_lines[DSR_index])
        height=float(height)
        height+=height_displacement 
        weight_list.append(float(SP_lines[DSR_index][SI_index])*float(SP2_lines[DSR_index]))
        z_list.append(float(height)) # midpoint
        r_list.append(float(rad))

real_r_lines=[float(line)-r_margin for line in DS3Q_lines]
real_r_lines.append(real_r_lines[-1]+r_margin*2) #openmc wants the corners, rather than the individual source points for the mesh

real_z_lines=[float(line)-z_margin for line in SI_lines]
real_z_lines.append(real_z_lines[-1]+z_margin*2)


r_grid = np.array(real_r_lines)
z_grid = np.array(real_z_lines)
phi_grid = np.array([0.0, 2*np.pi])# full angular coverage

cyl_mesh = openmc.CylindricalMesh(r_grid, z_grid, phi_grid, origin=(0, 0, height_displacement))
print(cyl_mesh)

In [ ]:
#reorder weight list: swap z and r iterations; depends if the order of array mesh array in MCNP matches already, always check!:
weight_list_rz=[0 for i in range(len(weight_list))]
r_pnts=102-1
z_pnts=212-1

for r in range(r_pnts):
    for z in range(z_pnts):
        weight_list_rz[z*r_pnts+r]=weight_list[r*z_pnts+z]
        

In [ ]:
astd=0.4/np.sqrt(2)*1e6 #Gaussian standard deviation in eV
Ecenter=14.1e6
src_rate=1e20
sources = []
norm1=sum(weight_list)
norm2=len(weight_list)

prepared_weightlist=[]
for weight in weight_list_rz:
    prepared_weightlist.append(src_rate*(weight/norm1)/norm2)

for weight in prepared_weightlist:
    mysrc=openmc.Source()
    mysrc.angle=openmc.stats.Isotropic()
    mysrc.energy = openmc.stats.Normal(Ecenter, astd)
    mysrc.strength=weight
    sources.append(mysrc)
mesh_src = openmc.MeshSource(cyl_mesh, sources)
mesh_src.set_total_strength(src_rate)


In [ ]:
settings = openmc.Settings()
settings.run_mode = 'fixed source'
settings.batches = 1000
settings.particles = 100000
settings.photon_transport=True
settings.source = mesh_src
filename='nodes'
settings.export_to_xml()